<a href="https://colab.research.google.com/github/bb88dd/hello-world/blob/master/Ben_Daly_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# NOTES FOR MY UNDERSTANDING OF GAME.
# Six gods / decks (war, nature, magic, light, deception and death)
# Each god has unique powers. Choose one power for your god at start of game.
# Each god has a unique set of 30 cards (can have max. 2 of each card)
# All decks have access to a shared pool of cards known as neutral cards.
# Mana is a resource required to play cards or god powers. It's refreshed at start of turn.
# Mana required to play card shown top left corner of card.
# Pronto: underlying stats for a class of card.

In [0]:
# Import libraries
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive

In [0]:
# Mount Google drive.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Function to return user info. for players with highest win rate.
def highest_win_rate_players(no_players, min_player_wins=50, records=150000, min_win_rate=0):
  URL = 'https://api.godsunchained.com/v0/properties?sort=won_matches&order=desc&perPage='
  response_properties = requests.get(URL + str(records))
  dict_properties = json.loads(response_properties.text)['records']  # Select records from dict. output.
  df_properties = pd.DataFrame(dict_properties)
  df_properties = df_properties[df_properties.won_matches >= min_player_wins]
  # Add total_matches and win_rate columns.
  df_properties['total_matches'] = df_properties.won_matches + df_properties.lost_matches
  df_properties['win_rate'] = df_properties.won_matches / df_properties.total_matches
  df_properties = df_properties[df_properties.win_rate >= min_win_rate]
  df_properties = df_properties.sort_values(by='win_rate', ascending=False)
  df_properties = df_properties.reset_index(drop=True)
  user_ids = df_properties.iloc[0:no_players, 2]
  usernames = df_properties.iloc[0:no_players, 3]
  return user_ids


In [0]:
# Function to return user deck info. for a list of user_ids
def decks_by_user_id(user_ids, records=50000):
  URL = 'https://api.godsunchained.com/v0/match?perPage='
  response_match = requests.get(URL + str(records))
  dict_match = json.loads(response_match.text)['records']
  df_match = json_normalize(dict_match, 'player_info')
  df_match = df_match[df_match.user_id.isin(user_ids)]
  df_match = df_match.reset_index(drop=True)
  return df_match


In [0]:
# Function to count gods frequency from dataframe of user deck info.
def gods_count(df):
  count = df['god'].value_counts()
  return count

In [0]:
# Function to count card frequency from dataframe of user deck info.
def card_freq(df, no_cards):
  cards_flat = pd.Series([item for sublist in df.cards for item in sublist])
  cards_count = cards_flat.value_counts().sort_index().rename_axis('cards').reset_index(name='freq').sort_values(by='freq', ascending=False)
  cards_count = cards_count.reset_index(drop=True)
  cards_count = cards_count.iloc[0:no_cards, :]
  return cards_count

In [0]:
# Function to barplot gods / decks.
def plot_gods(series,
              filename,
              title='Decks used by highest win rate players'):
  plt.clf()
  ax = sns.barplot(x=series.index.values, y=series)
  ax.set(ylabel='No. games played using deck',
         title=title)
  plt.savefig('/content/drive/My Drive/Jobs/Data Science/Gods_unchained/imgs/plot_gods_{}.png'.format(filename))
  #plt.show()
  return ax

In [0]:
# Function to barplot card frequency.
def plot_card_freq(df,
                   filename,
                   title='Most included cards across all decks with win rate > 50%'):
  plt.clf()
  ax = sns.barplot(x=df.cards, y=df.freq, order=df.cards)
  ax.set(xlabel='Card number',
         ylabel='No. times included',
         title=title)
  plt.savefig('/content/drive/My Drive/Jobs/Data Science/Gods_unchained/imgs/plot_cards_{}.png'.format(filename))
  #plt.show()
  return ax


#Viewing all decks used by the highest win-rate players
###This allows us to view what the best players are experimenting with.


In [0]:
# Get list of players with highest win rates (look at different min. wins)  
min_wins = [20, 50, 100]
user_ids_mw20 = highest_win_rate_players(no_players=10, min_player_wins=min_wins[0], min_win_rate=0.5)
user_ids_mw50 = highest_win_rate_players(no_players=10, min_player_wins=min_wins[1], min_win_rate=0.5)
user_ids_mw100 = highest_win_rate_players(no_players=10, min_player_wins=min_wins[2], min_win_rate=0.5)

In [0]:
# Dataframe with decks used top 10 win rates players are using
decks_mw20 = decks_by_user_id(user_ids_mw20)
decks_mw50 = decks_by_user_id(user_ids_mw50)
decks_mw100 = decks_by_user_id(user_ids_mw100)

In [0]:
# Count decks used by players with the highest win rate. 
deck_count_mw20 = gods_count(decks_mw20)
deck_count_mw50 = gods_count(decks_mw50)
deck_count_mw100 = gods_count(decks_mw100)

In [0]:
# Make plots for decks used by players with the highest win rate. 
sns.set(style='whitegrid')
plot_deck_mw20 = plot_gods(deck_count_mw20, filename='mw20', title='')
plot_deck_mw50 = plot_gods(deck_count_mw50, filename='mw50', title='')
plot_deck_mw100 = plot_gods(deck_count_mw100, filename='mw100', title='')


This analysis takes the ten players with the highest win rates and looks at which decks there are using. To be considered for the top ten players must have a minimum of 20 match wins. Figure 1 shows the number of games played per deck for the top 10 players.  

![test](https://docs.google.com/uc?export=download&id=1-38z6wzSOD5dl8VqFa2q1zjiHh53y_rb)  
*Figure 1: Decks used by the ten players with highest win rate (min. 20 wins)*  

The same analysis was carried out for looking at the top ten win rate players with a minimum of 50 and 100 wins (see Figures 2 and 3). The imbalance in decks increases for players who have taken part in more matches. It's possible that the war and nature decks have an advantage over the light, magic, death and deception decks.  

![min50](https://docs.google.com/uc?export=download&id=1--zjm9UwvYnlb0Z-CpmXr10xJxLRg-EL)  
*Figure 2: Decks used by the ten players with highest win rate (min. 50 wins)*  


![min100](https://docs.google.com/uc?export=download&id=1-10_5Y4C7XDZBk5N7xRRMjKRxYbKqzVr)  
*Figure 3: Decks used by the ten players with highest win rate (min. 100 wins)*




# Top 5 most included cards across all decks with above 50% win rate. 
### This allows us to track cards that are potentially warping a top level metagame.

In [0]:
# Plot top 5 most included cards across all decks for players with win rate ? 50%
user_ids = highest_win_rate_players(no_players=10000, min_player_wins=20, min_win_rate=0.5)
decks = decks_by_user_id(user_ids)
top5_cards = card_freq(decks, no_cards=5)
plot_top5_cards = plot_card_freq(top5_cards, filename='top5', title='')


In [0]:
# Plot top 10 most included cards across all decks for players with win rate ? 50%
user_ids = highest_win_rate_players(no_players=10000, min_player_wins=20, min_win_rate=0.5)
decks = decks_by_user_id(user_ids)
top5_cards = card_freq(decks, no_cards=10)
plot_top5_cards = plot_card_freq(top5_cards, filename='top10', title='')


This analysis examines the most cards most frequently included in players decks whose win rate is greater than 50%. Figure 4 shows the top 5 cards, figure 5 shows the top 10 cards to give some additional context.  

![top5](https://docs.google.com/uc?export=download&id=1-2zg_tWEqHL8ROmflcwdkbFMXdH5cJuW)  
*Figure 4: Five most include cards for players with win rate > 50%*  


![top10](https://docs.google.com/uc?export=download&id=13JsQlKWGoCUukEiFRJm6kACBNRDTrhQo)  
*Figure 5: Ten most include cards for players with win rate > 50%*
